In [1]:
import MDAnalysis as mda
from MDAnalysis.analysis import lineardensity as lin
import MDAnalysis.transformations as trans


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns


In [2]:
step_size=100

In [3]:
# Load simulations

class Peptide:

    def __init__(self, pep_name, xtc_file_path, tpr_file_path, peptide_number, amino_acid_count):
        
        self.xtc_file_path = xtc_file_path
        self.tpr_file_path = tpr_file_path
        self.peptide_number = peptide_number
        self.amino_acid_count = amino_acid_count
        self.u = mda.Universe(tpr_file_path, xtc_file_path)
        self.u.trajectory.add_transformations(trans.unwrap(self.u.select_atoms(f"backbone")))
        self.protein_atoms = self.u.select_atoms("protein")
        self.prot_residues = self.protein_atoms.residues
        self.res_names = self.prot_residues.resnames
        self.res_ids = self.prot_residues.residues.resids
        self.resid_maps = {i+1:j for i, j in enumerate(self.res_ids)}
        self.pep_dict = {
            k: ((k-1)*amino_acid_count+1, k*amino_acid_count) for k in range(1, peptide_number+1)
        }
        self.pep_dict = {k: (self.resid_maps[i[0]], self.resid_maps[i[1]]) for k,i in self.pep_dict.items()}

        self.start_id = self.pep_dict[1][0]
        self.end_id = self.pep_dict[1][1]
        self.pep_name = pep_name


    def load_traj(self):

        stop_sim = None
        start, stop_sim, _ = self.u.trajectory.check_slice_indices(None, None, None)
        frames = np.arange(start, stop_sim, step_size)
        n_frames = frames.size

        return frames, n_frames
    


In [6]:
peptide = Peptide(
    "WF2_8_1", 
    "/Volumes/miru_backup/pleu_conc/pleu8/pleu8/md_0_1.xtc",
    "/Volumes/miru_backup/pleu_conc/pleu8/pleu8/md_0_1.tpr",
    8, 25)


In [7]:

for pep_obj in [peptide]:    
    data = []
    u = pep_obj.u
    frames, n_frames = pep_obj.load_traj()
    for pep, res_id_range in tqdm(pep_obj.pep_dict.items()):
        backbone = u.select_atoms(f"resid {res_id_range[0]}:{res_id_range[1]} and backbone")
        nterm = u.select_atoms(f"resid {res_id_range[0]}:{res_id_range[1]} and name N")[0]
        cterm = u.select_atoms(f"resid {res_id_range[0]}:{res_id_range[1]} and name C")[-1]

        for frame_index, ts in enumerate(u.trajectory[frames]):
            r = cterm.position - nterm.position  # end-to-end vector from atom positions
            d = np.linalg.norm(r)   # end-to-end distance
            rgyr = backbone.radius_of_gyration()  # method of a AtomGroup; updates with each frame
            data.append((u.trajectory.time, pep, d, rgyr))

    df = pd.DataFrame(data, columns=["Time (ns)", "Peptide_num", "end_to_end_distance", "r_gyr"])
    df['Time (ns)'] = df['Time (ns)'].astype(float)/1000
    df['Time (ns)'] = df['Time (ns)'].astype(int)
    df.to_csv(f"rgyr_{pep_obj.pep_name}.csv")     

100%|██████████| 8/8 [00:07<00:00,  1.05it/s]
